In [1]:
import os

import numpy as np

import torch
import torch.nn as nn

from torch.autograd import Variable

import torchvision.utils

from data_loader import get_loader
from make_gif import make_gif

from DiscrimiatorModule import Discriminator
from GeneratorModule import Generator

In [2]:
def init_weights(m) :
    name = type(m)

    if name == nn.Conv3d or name == nn.ConvTranspose2d or name == nn.ConvTranspose3d :
        m.weight.data.normal_(0.0, 0.01)
        m.bias.data.fill_(0)
    elif name == nn.BatchNorm2d or name == nn.BatchNorm3d :
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [3]:
pre_train = True

batch_size = 64
video_size = 64
epoch_size = 1000
        
#check GPU
is_gpu = torch.cuda.is_available()
print(is_gpu)

if is_gpu :
    dtype = torch.cuda.FloatTensor
else :
    dtype = torch.FloatTensor

if pre_train :
    D = torch.load('./model/D2.ckpt').type(dtype)
    G = torch.load('./model/G2.ckpt').type(dtype)
else :
    D = Discriminator()
    D = D.type(dtype)

    G = Generator()
    G = G.type(dtype)

    D.apply(init_weights)
    G.apply(init_weights)

criterion = nn.BCEWithLogitsLoss().type(dtype)

d_optimizer = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))

True


In [4]:
data_loader = get_loader(data_path='./dataset', image_size=video_size, batch_size=batch_size, num_workers=2)

for epoch in range(640, epoch_size + 1) :
    for iter, (video, y) in enumerate(data_loader) :
        local_batch_size = video.size()[0]
        
        real_labels = Variable(torch.ones(local_batch_size, 2).type(dtype))
        fake_labels = Variable(torch.zeros(local_batch_size, 2).type(dtype))
        
        # 1. Train Discriminator
        video_data = Variable(video).type(dtype)
        y = Variable(y).type(dtype)
        
        y_data = torch.unsqueeze(torch.unsqueeze(torch.unsqueeze(y, -1), -1), -1)
        y_data = y_data.expand(local_batch_size, 6, 32, 64, 64)
       
        
        
        # 1-1. Real Video
        outputs = D(video_data, y_data).view(local_batch_size, 2)
        d_loss_real = criterion(outputs, real_labels)

        
        
        # 1-2. Fake Video
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        
        D.zero_grad()
        G.zero_grad()
        d_loss.backward()
        d_optimizer.step()





        # 2. Train Generator
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)

        g_loss = criterion(outputs, real_labels)
        D.zero_grad()
        G.zero_grad()

        g_loss.backward()
        g_optimizer.step()
          
        print('Epoch [%d/%d], Iter [%d/%d], d_loss: %.4f, g_loss: %.4f' % (epoch, epoch_size, iter, len(data_loader), d_loss.data[0], g_loss.data[0]))
    
    print('Model saving...')
    
    if epoch % 100 == 0 :
        torch.save(D, './model/D_' + str(epoch) + '.ckpt')
        torch.save(G, './model/G_' + str(epoch) + '.ckpt')
    else :  
        torch.save(D, './model/D2.ckpt')
        torch.save(G, './model/G2.ckpt')

Epoch [450/1000], Iter [0/49], d_loss: 0.0094, g_loss: 10.3531
Epoch [450/1000], Iter [1/49], d_loss: 0.2137, g_loss: 6.6421
Epoch [450/1000], Iter [2/49], d_loss: 0.0568, g_loss: 7.0265
Epoch [450/1000], Iter [3/49], d_loss: 0.0821, g_loss: 8.7194
Epoch [450/1000], Iter [4/49], d_loss: 0.0086, g_loss: 6.7944
Epoch [450/1000], Iter [5/49], d_loss: 0.0310, g_loss: 7.7140
Epoch [450/1000], Iter [6/49], d_loss: 0.0050, g_loss: 8.6255
Epoch [450/1000], Iter [7/49], d_loss: 0.0111, g_loss: 7.0106
Epoch [450/1000], Iter [8/49], d_loss: 0.0254, g_loss: 6.6630
Epoch [450/1000], Iter [9/49], d_loss: 0.0508, g_loss: 6.9895
Epoch [450/1000], Iter [10/49], d_loss: 0.0911, g_loss: 3.6451
Epoch [450/1000], Iter [11/49], d_loss: 0.0357, g_loss: 5.4329
Epoch [450/1000], Iter [12/49], d_loss: 0.1991, g_loss: 11.5856
Epoch [450/1000], Iter [13/49], d_loss: 0.0074, g_loss: 13.8796
Epoch [450/1000], Iter [14/49], d_loss: 0.0122, g_loss: 14.8568
Epoch [450/1000], Iter [15/49], d_loss: 0.0812, g_loss: 11.72

Epoch [452/1000], Iter [32/49], d_loss: 0.0083, g_loss: 5.2368
Epoch [452/1000], Iter [33/49], d_loss: 0.0265, g_loss: 7.0186
Epoch [452/1000], Iter [34/49], d_loss: 0.0652, g_loss: 7.8535
Epoch [452/1000], Iter [35/49], d_loss: 0.0043, g_loss: 7.2767
Epoch [452/1000], Iter [36/49], d_loss: 0.0436, g_loss: 6.1449
Epoch [452/1000], Iter [37/49], d_loss: 0.0060, g_loss: 7.0638
Epoch [452/1000], Iter [38/49], d_loss: 0.0455, g_loss: 9.1470
Epoch [452/1000], Iter [39/49], d_loss: 0.0378, g_loss: 7.6863
Epoch [452/1000], Iter [40/49], d_loss: 0.0119, g_loss: 7.3071
Epoch [452/1000], Iter [41/49], d_loss: 0.0053, g_loss: 7.3153
Epoch [452/1000], Iter [42/49], d_loss: 0.0054, g_loss: 6.3976
Epoch [452/1000], Iter [43/49], d_loss: 0.0193, g_loss: 6.2467
Epoch [452/1000], Iter [44/49], d_loss: 0.0067, g_loss: 9.7529
Epoch [452/1000], Iter [45/49], d_loss: 0.1010, g_loss: 5.0162
Epoch [452/1000], Iter [46/49], d_loss: 0.2860, g_loss: 10.3567
Epoch [452/1000], Iter [47/49], d_loss: 0.0826, g_loss

Epoch [455/1000], Iter [15/49], d_loss: 0.0228, g_loss: 7.4324
Epoch [455/1000], Iter [16/49], d_loss: 0.0688, g_loss: 6.5772
Epoch [455/1000], Iter [17/49], d_loss: 0.0594, g_loss: 7.1627
Epoch [455/1000], Iter [18/49], d_loss: 0.0117, g_loss: 6.4304
Epoch [455/1000], Iter [19/49], d_loss: 0.0239, g_loss: 6.8131
Epoch [455/1000], Iter [20/49], d_loss: 0.0165, g_loss: 6.1552
Epoch [455/1000], Iter [21/49], d_loss: 0.0518, g_loss: 8.0033
Epoch [455/1000], Iter [22/49], d_loss: 0.0251, g_loss: 5.2223
Epoch [455/1000], Iter [23/49], d_loss: 0.0363, g_loss: 4.4601
Epoch [455/1000], Iter [24/49], d_loss: 0.0107, g_loss: 4.3777
Epoch [455/1000], Iter [25/49], d_loss: 0.0854, g_loss: 7.4276
Epoch [455/1000], Iter [26/49], d_loss: 0.2608, g_loss: 5.2648
Epoch [455/1000], Iter [27/49], d_loss: 0.0231, g_loss: 3.3850
Epoch [455/1000], Iter [28/49], d_loss: 0.2082, g_loss: 7.3130
Epoch [455/1000], Iter [29/49], d_loss: 0.0121, g_loss: 8.2195
Epoch [455/1000], Iter [30/49], d_loss: 0.2066, g_loss:

Epoch [457/1000], Iter [47/49], d_loss: 0.0023, g_loss: 13.2081
Epoch [457/1000], Iter [48/49], d_loss: 0.0288, g_loss: 12.8885
Model saving...
Epoch [458/1000], Iter [0/49], d_loss: 0.0204, g_loss: 10.7818
Epoch [458/1000], Iter [1/49], d_loss: 0.0094, g_loss: 10.0578
Epoch [458/1000], Iter [2/49], d_loss: 0.0217, g_loss: 6.5746
Epoch [458/1000], Iter [3/49], d_loss: 0.0599, g_loss: 7.6293
Epoch [458/1000], Iter [4/49], d_loss: 0.0006, g_loss: 8.3286
Epoch [458/1000], Iter [5/49], d_loss: 0.0279, g_loss: 6.6893
Epoch [458/1000], Iter [6/49], d_loss: 0.0059, g_loss: 7.4195
Epoch [458/1000], Iter [7/49], d_loss: 0.0354, g_loss: 8.5349
Epoch [458/1000], Iter [8/49], d_loss: 0.0574, g_loss: 8.2705
Epoch [458/1000], Iter [9/49], d_loss: 0.0272, g_loss: 5.7917
Epoch [458/1000], Iter [10/49], d_loss: 0.0253, g_loss: 4.2309
Epoch [458/1000], Iter [11/49], d_loss: 0.0442, g_loss: 5.9294
Epoch [458/1000], Iter [12/49], d_loss: 0.0955, g_loss: 8.3776
Epoch [458/1000], Iter [13/49], d_loss: 0.026

Epoch [460/1000], Iter [30/49], d_loss: 0.0275, g_loss: 6.9877
Epoch [460/1000], Iter [31/49], d_loss: 0.0471, g_loss: 6.9226
Epoch [460/1000], Iter [32/49], d_loss: 0.0258, g_loss: 9.3263
Epoch [460/1000], Iter [33/49], d_loss: 0.2661, g_loss: 5.1559
Epoch [460/1000], Iter [34/49], d_loss: 0.1070, g_loss: 6.0608
Epoch [460/1000], Iter [35/49], d_loss: 0.0750, g_loss: 8.5158
Epoch [460/1000], Iter [36/49], d_loss: 0.0031, g_loss: 10.6321
Epoch [460/1000], Iter [37/49], d_loss: 0.0042, g_loss: 9.3072
Epoch [460/1000], Iter [38/49], d_loss: 0.0801, g_loss: 6.6213
Epoch [460/1000], Iter [39/49], d_loss: 0.0102, g_loss: 6.3020
Epoch [460/1000], Iter [40/49], d_loss: 0.0167, g_loss: 5.7308
Epoch [460/1000], Iter [41/49], d_loss: 0.0181, g_loss: 6.0506
Epoch [460/1000], Iter [42/49], d_loss: 0.1266, g_loss: 8.7280
Epoch [460/1000], Iter [43/49], d_loss: 0.0113, g_loss: 10.1189
Epoch [460/1000], Iter [44/49], d_loss: 0.1453, g_loss: 9.3711
Epoch [460/1000], Iter [45/49], d_loss: 0.0076, g_los

...

Epoch [633/1000], Iter [3/49], d_loss: 0.0293, g_loss: 7.5121
Epoch [633/1000], Iter [4/49], d_loss: 0.0041, g_loss: 7.9670
Epoch [633/1000], Iter [5/49], d_loss: 0.0533, g_loss: 6.2863
Epoch [633/1000], Iter [6/49], d_loss: 0.0095, g_loss: 5.1075
Epoch [633/1000], Iter [7/49], d_loss: 0.0071, g_loss: 4.4906
Epoch [633/1000], Iter [8/49], d_loss: 0.0045, g_loss: 5.6207
Epoch [633/1000], Iter [9/49], d_loss: 0.0677, g_loss: 6.6053
Epoch [633/1000], Iter [10/49], d_loss: 0.0356, g_loss: 6.6993
Epoch [633/1000], Iter [11/49], d_loss: 0.0816, g_loss: 7.8571
Epoch [633/1000], Iter [12/49], d_loss: 0.0219, g_loss: 8.7000
Epoch [633/1000], Iter [13/49], d_loss: 0.0043, g_loss: 12.6813
Epoch [633/1000], Iter [14/49], d_loss: 0.0228, g_loss: 5.8606
Epoch [633/1000], Iter [15/49], d_loss: 0.0025, g_loss: 7.1673
Epoch [633/1000], Iter [16/49], d_loss: 0.0166, g_loss: 5.3973
Epoch [633/1000], Iter [17/49], d_loss: 0.0922, g_loss: 5.1245
Epoch [633/1000], Iter [18/49], d_loss: 0.2059, g_loss: 1.718

Epoch [635/1000], Iter [35/49], d_loss: 0.0361, g_loss: 16.2424
Epoch [635/1000], Iter [36/49], d_loss: 0.0006, g_loss: 13.6301
Epoch [635/1000], Iter [37/49], d_loss: 0.0052, g_loss: 13.4640
Epoch [635/1000], Iter [38/49], d_loss: 0.0304, g_loss: 13.3331
Epoch [635/1000], Iter [39/49], d_loss: 0.0114, g_loss: 11.0127
Epoch [635/1000], Iter [40/49], d_loss: 0.0106, g_loss: 11.0725
Epoch [635/1000], Iter [41/49], d_loss: 0.0058, g_loss: 8.3053
Epoch [635/1000], Iter [42/49], d_loss: 0.0007, g_loss: 10.4656
Epoch [635/1000], Iter [43/49], d_loss: 0.0184, g_loss: 8.4104
Epoch [635/1000], Iter [44/49], d_loss: 0.0014, g_loss: 9.3322
Epoch [635/1000], Iter [45/49], d_loss: 0.0225, g_loss: 8.1721
Epoch [635/1000], Iter [46/49], d_loss: 0.0032, g_loss: 8.1426
Epoch [635/1000], Iter [47/49], d_loss: 0.0220, g_loss: 8.3714
Epoch [635/1000], Iter [48/49], d_loss: 0.8024, g_loss: 6.6793
Model saving...
Epoch [636/1000], Iter [0/49], d_loss: 1.3397, g_loss: 15.7625
Epoch [636/1000], Iter [1/49], d

Epoch [638/1000], Iter [18/49], d_loss: 0.0167, g_loss: 4.7201
Epoch [638/1000], Iter [19/49], d_loss: 0.0091, g_loss: 5.6684
Epoch [638/1000], Iter [20/49], d_loss: 0.0073, g_loss: 7.0727
Epoch [638/1000], Iter [21/49], d_loss: 0.0125, g_loss: 7.4655
Epoch [638/1000], Iter [22/49], d_loss: 0.0064, g_loss: 6.0939
Epoch [638/1000], Iter [23/49], d_loss: 0.0088, g_loss: 6.5037
Epoch [638/1000], Iter [24/49], d_loss: 0.0017, g_loss: 6.0318
Epoch [638/1000], Iter [25/49], d_loss: 0.0163, g_loss: 6.3202
Epoch [638/1000], Iter [26/49], d_loss: 0.0767, g_loss: 9.5097
Epoch [638/1000], Iter [27/49], d_loss: 0.0129, g_loss: 8.2696
Epoch [638/1000], Iter [28/49], d_loss: 0.0144, g_loss: 7.4565
Epoch [638/1000], Iter [29/49], d_loss: 0.0126, g_loss: 7.7459
Epoch [638/1000], Iter [30/49], d_loss: 0.0094, g_loss: 6.8278
Epoch [638/1000], Iter [31/49], d_loss: 0.0111, g_loss: 6.2003
Epoch [638/1000], Iter [32/49], d_loss: 0.0174, g_loss: 8.0437
Epoch [638/1000], Iter [33/49], d_loss: 0.0028, g_loss:

Process Process-382:
Process Process-381:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dskym0/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dskym0/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dskym0/anaconda3/l

KeyboardInterrupt: 